<a href="https://colab.research.google.com/github/YusunPark/move_bike/blob/main/%EC%9E%84%EC%8B%9C_%EC%8B%A4%EC%8B%9C%EA%B0%84_%EB%8D%B0%EC%9D%B4%ED%84%B0_%EC%83%9D%EC%84%B1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import time
import requests
import json
import pandas as pd
import numpy as np
import datetime

In [ ]:
# 날씨 호출

def call_weather():
  lon = 126.9896
  lat = 37.5335
  WEATHER_API_KEY = '3d3df4ac693138f8157192b265f0844e'

  weather_url = f'https://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&appid={WEATHER_API_KEY}&units=metric'
  resp = requests.get(weather_url)
  weather_json = json.loads(resp.text)
  return weather_json

In [ ]:
# 자전거 데이터 호출
def call_bike():
  BIKE_API_KEY = '74796e6d72736f6c39314a50736843'

  bike_url_1 = f'http://openapi.seoul.go.kr:8088/{BIKE_API_KEY}/json/bikeList/1/1000/'
  bike_url_2 = f'http://openapi.seoul.go.kr:8088/{BIKE_API_KEY}/json/bikeList/1001/2000/'
  bike_url_3 = f'http://openapi.seoul.go.kr:8088/{BIKE_API_KEY}/json/bikeList/2001/3000/'

  resp_bike_1 = requests.get(bike_url_1)
  bike_json_1 = json.loads(resp_bike_1.text)

  resp_bike_2 = requests.get(bike_url_2)
  bike_json_2 = json.loads(resp_bike_2.text)

  resp_bike_3 = requests.get(bike_url_3)
  bike_json_3 = json.loads(resp_bike_3.text)

  return bike_json_1, bike_json_2, bike_json_3

In [ ]:
# 초기 실행

# bike_json 생성을 위해 초기 1번은 그냥 함
weather_json = call_weather()
bike_json_1, bike_json_2, bike_json_3 = call_bike()

# 날씨
date = []
temp = []
feels_like = []
main = []
wind_speed = []

date.append(str(datetime.datetime.now()))             # 날짜
main.append(weather_json['weather'][0]['main'])       # 흐림/ 맑음 등
temp.append(weather_json['main']['temp'])             # 기온
feels_like.append(weather_json['main']['feels_like']) # 체감온도
wind_speed.append(weather_json['wind']['speed'])      # 풍속



# 자전거
df1 = pd.DataFrame.from_dict(bike_json_1['rentBikeStatus']['row'])
df2 = pd.DataFrame.from_dict(bike_json_2['rentBikeStatus']['row'])
df3 = pd.DataFrame.from_dict(bike_json_3['rentBikeStatus']['row'])

bike_data = pd.concat([df1, df2, df3])
bike_data['time'] = str(datetime.datetime.now())


,rackTotCnt,stationName,parkingBikeTotCnt,shared,stationLatitude,stationLongitude,stationId,time
0,68,위트콤공장,0,0,37.48421478,126.99629211,ST-598,2022-03-29 09:36:13.723577
1,22,102. 망원역 1번출구 앞,20,91,37.55564880,126.91062927,ST-4,2022-03-29 09:36:13.723577
2,16,103. 망원역 2번출구 앞,7,44,37.55495071,126.91083527,ST-5,2022-03-29 09:36:13.723577
3,15,104. 합정역 1번출구 앞,15,100,37.55062866,126.91498566,ST-6,2022-03-29 09:36:13.723577
4,7,105. 합정역 5번출구 앞,6,86,37.55000687,126.91482544,ST-7,2022-03-29 09:36:13.723577
...,...,...,...,...,...,...,...,...
590,4,5301. 상계백병원점,3,75,37.64805984,127.06239319,ST-3015,2022-03-29 09:36:13.723577
591,15,5851. 도림천 건널목 위,10,67,37.50208282,126.89318848,ST-3040,2022-03-29 09:36:13.723577
592,7,5852. 대림동현대2차 201동 앞,2,29,37.50300598,126.89382935,ST-3041,2022-03-29 09:36:13.723577
593,4,상담센터,1,25,37.57164001,127.03565979,ST-1747,2022-03-29 09:36:13.723577


In [ ]:
# 반복 10분(600초) / 48시간(288)
count = 144

while(count):
  time.sleep(600)
  print(f"loading..{count}")

  weather_json = call_weather()
  bike_json_1, bike_json_2, bike_json_3 = call_bike()

  date.append(str(datetime.datetime.now()))             # 날짜
  main.append(weather_json['weather'][0]['main'])       # 흐림/ 맑음 등
  temp.append(weather_json['main']['temp'])             # 기온
  feels_like.append(weather_json['main']['feels_like']) # 체감온도
  wind_speed.append(weather_json['wind']['speed'])      # 풍속

  # 2번째부턴 원래에 이어붙이기
  df1 = pd.DataFrame.from_dict(bike_json_1['rentBikeStatus']['row'])
  df2 = pd.DataFrame.from_dict(bike_json_2['rentBikeStatus']['row'])
  df3 = pd.DataFrame.from_dict(bike_json_3['rentBikeStatus']['row'])

  df1['time'] = str(datetime.datetime.now())
  df2['time'] = str(datetime.datetime.now())
  df3['time'] = str(datetime.datetime.now())

  bike_data = pd.concat([bike_data, df1, df2, df3])
  count -= 1
  

# 작업 종료 시
weather_data = pd.DataFrame({'date': date, 'main':main, 'temp':temp, 'feels_like':feels_like, 'wind_speed':wind_speed} )

# 파일로 저장
bike_data.to_csv("bike_data.csv")
weather_data.to_csv("weather_data.csv")

loading..2
loading..1
